# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are:

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [ ]:
import nltk
bigram_counts = []
def read_bigrams(bigram_path):
    with open(bigram_path, "r", encoding="latin-1") as file:
        for line in file:
            parts = line.strip().split("\t")
            if len(parts) == 3:
                count = int(parts[0])
                w1, w2 = parts[1], parts[2]
                bigram_counts.append((count, w1,w2))

read_bigrams("bigrams.txt")
print(bigram_counts[:10])

[(275, 'a', 'a'), (31, 'a', 'aaa'), (29, 'a', 'all'), (45, 'a', 'an'), (192, 'a', 'and'), (39, 'a', 'another'), (25, 'a', 'at'), (82, 'a', 'b'), (45, 'a', 'b+'), (26, 'a', 'b-17')]


In [ ]:
bigram_dict = {(word1, word2): count for count, word1, word2 in bigram_counts}


In [ ]:
from collections import defaultdict

# number of situation when the 1st word of a bigram appear
word1_counts = defaultdict(int)
for count, word1, _ in bigram_counts:
    word1_counts[word1] += count

# compute conditional probabilities for bigrams
bigram_probs = { (w1, w2): count / word1_counts[w1] for (w1, w2), count in bigram_dict.items() }


In [ ]:
import math

# keyboard layout with coordinates to caclulate distances so that take into account typos
keyboard = {
    'q': (0, 0), 'w': (1, 0), 'e': (2, 0), 'r': (3, 0), 't': (4, 0),
    'y': (5, 0), 'u': (6, 0), 'i': (7, 0), 'o': (8, 0), 'p': (9, 0),
    'a': (0, 1), 's': (1, 1), 'd': (2, 1), 'f': (3, 1), 'g': (4, 1),
    'h': (5, 1), 'j': (6, 1), 'k': (7, 1), 'l': (8, 1),
    'z': (0, 2), 'x': (1, 2), 'c': (2, 2), 'v': (3, 2), 'b': (4, 2),
    'n': (5, 2), 'm': (6, 2)
}

def calulate_euclidean_distance(w1, w2):
    x1, y1 = keyboard.get(w1, (0, 0))
    x2, y2 = keyboard.get(w2, (0, 0))
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def my_levenshtein_distance(s1, s2):
    len_s1 = len(s1)
    len_s2 = len(s2)
    dp = [[0] * (len_s2 + 1) for _ in range(len_s1 + 1)]
    for i in range(len_s1 + 1):
        dp[i][0] = i
    for j in range(len_s2 + 1):
        dp[0][j] = j

    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            if s1[i - 1] == s2[j - 1]:
                cost = 0
            else:
                cost = calulate_euclidean_distance(s1[i - 1], s2[j - 1]) / 2  # some penalty
            dp[i][j] = min(
                dp[i - 1][j] + 1,
                dp[i][j - 1] + 1,
                dp[i - 1][j - 1] + cost
            )
    return dp[len_s1][len_s2]
# s1 = "gyu"
# s2 = "guy"
# distance = my_levenshtein_distance(s1, s2)
# print(distance)

QWERTY-aware Levenshtein distance between 'gyu' and 'guy': 1.0


In [ ]:
# generate my vocab for this task
keys = bigram_dict.keys()
unique_words = {word for tuple_pair in keys for word in tuple_pair}
print(len(unique_words))
unique_words = list(unique_words)

68784


In [ ]:
import numpy as np
# function to obtain the substitution
def get_best_candidate(context_word, incorrect_word, k=30):
  distances = []
  # calculate distance between incorrect word and all words to retrieve K nearest words from vocab
  for i in range(len(unique_words)):
    dist = my_levenshtein_distance(unique_words[i], incorrect_word)
    distances.append(dist)
  distances = np.array(distances)
  # print(distances[0])
  top_ind = np.argpartition(distances, k)[:k]
  # print(distances[0])
  # print(distances[top_ind[0]])
  # print(top_ind)
  # calculate the rating for the obtained on the previous step candidates. Score is bigram_prob divided by distance (less distance is better, therefore divide)
  scores = []
  for i in range(len(top_ind)):
    # print(bigram_probs.get((context_word, unique_words[top_ind[i]])))
    if bigram_probs.get((context_word, unique_words[top_ind[i]])) is None:
      # print(bigram_probs.get((context_word, unique_words[top_ind[i]])))11
      bigram_probs[(context_word, unique_words[top_ind[i]])] = 1e-8
    bigram_prob = bigram_probs.get((context_word, unique_words[top_ind[i]]))
    score = bigram_prob / distances[top_ind[i]]
    scores.append(score)
  scores = np.array(scores)
  # print(scores)
  # print(scores[:5])
  max_score_index = np.argmax(scores)
  word = unique_words[top_ind[max_score_index]]
  # for i in range(len(top_ind)):
    # print(unique_words[top_ind[i]], scores[i])
  # print(word)
  return word

In [ ]:
get_best_candidate("mobile", "pohne")

'phone'

In [ ]:
sentence = "i am the best gyu"
sentence_splitted = sentence.split(" ")
sentence_mask = []
for word in sentence_splitted:
  if word in unique_words:
    sentence_mask.append(True)
  else:
    sentence_mask.append(False)

print(sentence_mask)

[True, True, True, True, False]


In [ ]:
# I use mask to correct only the words with the mistakes to avoid redundant typo fixes which will make right word wrong:)
def check_and_correct_sentence(sentence):
  sentence_splitted = sentence.split(" ")
  sentence_mask = []
  for word in sentence_splitted:
    if word in unique_words:
      sentence_mask.append(True)
    else:
      sentence_mask.append(False)
  # print(sentence_splitted)

  for i in range(len(sentence_mask)):
    if sentence_mask[i] == False:
      word = get_best_candidate(sentence_splitted[i - 1], sentence_splitted[i])
      # print(word)
      sentence_splitted[i] =  word
      # print(sentence_splitted)
  # print(sentence_splitted)
  new_sentence = " ".join(sentence_splitted)
  # print(new_sentence)
  return new_sentence

In [ ]:
sentence = "i am the best gyu"
check_and_correct_sentence(sentence)

'i am the best buy'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

*Your text here...*

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [ ]:
# Your code here
# I just copy 10 sentences from the different websites in the internet.
# https://www.twinkl.com/teaching-wiki/descriptive-writing
# https://www.thoughtco.com/model-descriptive-paragraphs-1690573
orig_sentences = [
    "My most valuable possession is an old, slightly warped blond guitar.",
    "His pride, however, does not extend to his appearance, for he spends most of his time indoors watching television and growing fat.",
    "Notice how many different descriptors go into just the sentence about how the cat walks.",
    "The positioning of items relative to other items is on full display in this paragraph, to give people a clear vision of the layout of the place as a whole.",
    "Observe how the writer moves clearly from a description of the head of the clown to the body to the unicycle underneath",
    "The blond wood has been chipped and gouged to gray, particularly where the pick guard fell off years ago",
    "Instead of the perfect summer day of five minutes ago, the sky was darker than the blackest night. Not only had the sun disappeared, there weren't even any stars.",
    "Instead of accepting my plan, the council decided to sell the site to a property developer who will knock it down and build a load of luxury apartments.",
    "This definition of descriptive language could even be used in your everyday speech",
    "Story writing is usually the kind of writing that people associate with descriptive writing, and not without good reason"
    ]

# And I just create 5 sentences with errors for every one correct sentence. 4 with single error and one with errors in all words
sentences_with_errors = [[ "My most valuble possession is an old, slightly warped blond guitar.", "My most valuable possesion is an old, slightly warped blond guitar.", "My most valuable possession is an old, slightly warped blodn guitar.", "My most valuable possession is an old, slightly warped blond guiatr.", "My most valuble possesion is an old, slightly warped blodn guiatr."],
                         ["His prid, however, does not extend to his appearance, for he spends most of his time indoors watching television and growing fat.", "His pride, however, does not extend to his apearance, for he spends most of his time indoors watching television and growing fat.", "His pride, however, does not extend to his appearance, for he spends most of his time indors watching television and growing fat.", "His pride, however, does not extend to his appearance, for he spends most of his time indoors watching televison and growing fat.", "His prid, however, does not extend to his apearance, for he spends most of his time indors watching televison and growing fat."],
                         ["Notice how many different descripters go into just the sentence about how the cat walks.", "Notice how many different descriptors go into just the sentance about how the cat walks.", "Notice how many different descriptors go into just the sentence about how the cat wlaks.", "Notice how many different descriptors go into just the sentence about how the cat walks.", "Notice how many diffrent descripters go into just the sentance about how the cat wlaks."],
                         ["The positioing of items relative to other items is on full display in this paragraph, to give people a clear vision of the layout of the place as a whole.", "The positioning of items relative to other items is on full dispaly in this paragraph, to give people a clear vision of the layout of the place as a whole.", "The positioning of items relative to other items is on full display in this paragrpah, to give people a clear vision of the layout of the place as a whole.", "The positioning of items relative to other items is on full display in this paragraph, to give people a clear vison of the layout of the place as a whole.", "The positioing of items relative to other items is on full dispaly in this paragrpah, to give people a clear vison of the layout of the place as a whole."],
                         [" Observe how the writer moves clealry from a description of the head of the clown to the body to the unicycle underneath.", " Observe how the writer moves clearly from a descripton of the head of the clown to the body to the unicycle underneath.", " Observe how the writer moves clearly from a description of the head of the clow to the body to the unicycle underneath.", " Observe how the writer moves clearly from a description of the head of the clown to the body to the unicycle underneat.", " Observe how the writter moves clealry from a descripton of the head of the clow to the body to the unicycle underneat."],
                         ["The blodn wood has been chipped and gouged to gray, particularly where the pick guard fell off years ago.", "The blond wood has been chipped and gouged to gray, particulary where the pick guard fell off years ago.","The blond wood has been chipped and gouged to gray, particularly where the pick gaurd fell off years ago.", "The blond wood has been chipped and gouged to gray, particularly where the pick guard fel off years ago.", "The blodn wood has been chipped and gouged to gray, particulary where the pick gaurd fel off years ago."],
                         ["Instead of the perfect sumer day of five minutes ago, the sky was darker than the blackest night. Not only had the sun disappeared, there weren't even any stars.", "Instead of the perfect summer day of five minuts ago, the sky was darker than the blackest night. Not only had the sun disappeared, there weren't even any stars.", "Instead of the perfect summer day of five minutes ago, the sky was darker than the blackest night. Not only had the sun disapeared, there weren't even any stars.", "Instead of the perfect summer day of five minutes ago, the sky was darker than the blackest night. Not only had the sun disappeared, there werent even any stars.", "Instead of the perfect sumer day of five minuts ago, the sky was darker than the blackest night. Not only had the sun disapeared, there werent even any stars."],
                         ["Instead of acepting my plan, the council decided to sell the site to a property developer who will knock it down and build a load of luxury apartments.", "Instead of accepting my plan, the council decided to sell the site to a property develper who will knock it down and build a load of luxury apartments.", "Instead of accepting my plan, the council decided to sell the site to a property developer who will knock it down and bild a load of luxury apartments.", "Instead of accepting my plan, the council decided to sell the site to a property developer who will knock it down and build a load of luxury apartmets.", "Instead of acepting my plan, the council decided to sell the site to a property develper who will knock it down and bild a load of luxury apartmets."],
                         ["This definiton of descriptive language could even be used in your everyday speech.", "This definition of descriptive langauge could even be used in your everyday speech.", "This definition of descriptive language could even be used in your everyday speach.", "This definition of descriptive language could even be used in your everydy speech.", "This definiton of descriptive langauge could even be used in your everydy speach."],
                         ["Story writing is usually the kind of writing that people asociate with descriptive writing, and not without good reason.", "Story writing is usually the kind of writing that people associate with descriptive writing, and not without good reaosn.", "Story writing is usually the kind of writing that peple associate with descriptive writing, and not without good reason.", "Story writing is usually the kind of writing that people associate with descriptive writing, ant not without good reason.", "Story writting is usually the kind of writing that peple asociate with descriptive writing, ant not without good reaosn."],
                         ]


In [ ]:
import string
translator = str.maketrans('', '', string.punctuation)

# Remove punctuation
def convert_sentence_to_proper_format(sentece_to_clean):
  # print(sentece_to_clean)
  sentece_to_clean = sentece_to_clean.strip()
  sentece_to_clean = sentece_to_clean.lower()
  clean_text = sentece_to_clean.translate(translator)
  # print(clean_text)
  return clean_text

In [ ]:
clean_sentences_with_errors = []
for i in range(len(sentences_with_errors)):
  new_list = []
  for j in range(len(sentences_with_errors[i])):
    new_list.append(convert_sentence_to_proper_format(sentences_with_errors[i][j]))
  clean_sentences_with_errors.append(new_list)
print(clean_sentences_with_errors)

[['my most valuble possession is an old slightly warped blond guitar', 'my most valuable possesion is an old slightly warped blond guitar', 'my most valuable possession is an old slightly warped blodn guitar', 'my most valuable possession is an old slightly warped blond guiatr', 'my most valuble possesion is an old slightly warped blodn guiatr'], ['his prid however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his apearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching televison and growing fat', 'his prid however does not extend to his apearance for he spends most of his time indors watching televison and growing fat'], ['notic

In [ ]:
all_corr_sent = []
for i in range(len(clean_sentences_with_errors)):
  corr_sent = []
  for j in range(len(clean_sentences_with_errors[i])):
    corrected_sentence = check_and_correct_sentence(clean_sentences_with_errors[i][j])
    corr_sent.append(corrected_sentence)
  all_corr_sent.append(corr_sent)
print(all_corr_sent)

[['my most valuable possession is an old slightly warped blond guitar', 'my most valuable possession is an old slightly warped blond guitar', 'my most valuable possession is an old slightly warped blown guitar', 'my most valuable possession is an old slightly warped blond guitar', 'my most valuable possession is an old slightly warped blown guitar'], ['his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fa

In [ ]:
converted_orig_sentences = []
for i in range(len(orig_sentences)):
    converted_orig_sentences.append(convert_sentence_to_proper_format(orig_sentences[i]))
print(converted_orig_sentences)

['my most valuable possession is an old slightly warped blond guitar', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'notice how many different descriptors go into just the sentence about how the cat walks', 'the positioning of items relative to other items is on full display in this paragraph to give people a clear vision of the layout of the place as a whole', 'observe how the writer moves clearly from a description of the head of the clown to the body to the unicycle underneath', 'the blond wood has been chipped and gouged to gray particularly where the pick guard fell off years ago', 'instead of the perfect summer day of five minutes ago the sky was darker than the blackest night not only had the sun disappeared there werent even any stars', 'instead of accepting my plan the council decided to sell the site to a property developer who will knock it down and build a load of luxury apartments', 'this 

In [ ]:
count = 0
for i in range(len(clean_sentences_with_errors)):
  for j in range(len(clean_sentences_with_errors[i])):
    if converted_orig_sentences[i] == all_corr_sent[i][j]:
      count += 1
      print(all_corr_sent[i][j])
print("My dumb spellchecker accuracy",count/50)

my most valuable possession is an old slightly warped blond guitar
my most valuable possession is an old slightly warped blond guitar
my most valuable possession is an old slightly warped blond guitar
his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat
his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat
his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat
his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat
his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat
notice how many different descriptors go into just the sentence about how the cat walks
notice how many different descriptors go into just the sentence about how th

Norvig corrector


In [ ]:
# just copy-paste from  https://norvig.com/spell-correct.html
import re
from collections import Counter

def words(text):
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
def correct_sentence_norvig(sentence):
    return ' '.join(correction(word) for word in sentence.split())
norvig_corrected_sentences = []
for i in range(len(clean_sentences_with_errors)):
  norvig_one_sent = []
  for j in range(len(clean_sentences_with_errors[i])):
    # print(i, j)
    # print(clean_sentences_with_errors[i][j])
    norvig_one_sent.append(correct_sentence_norvig(clean_sentences_with_errors[i][j]))
  norvig_corrected_sentences.append(norvig_one_sent)
print(norvig_corrected_sentences)

[['my most valuable possession is an old slightly warped blood guitar', 'my most valuable possession is an old slightly warped blood guitar', 'my most valuable possession is an old slightly warped blown guitar', 'my most valuable possession is an old slightly warped blood guitar', 'my most valuable possession is an old slightly warped blown guitar'], ['his paid however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his paid however does not extend to his appearance for he spends most of his time indoors watching television and growing fat'

In [ ]:
count_norvig = 0
for i in range(len(norvig_corrected_sentences)):
  for j in range(len(norvig_corrected_sentences[i])):
    if converted_orig_sentences[i] == norvig_corrected_sentences[i][j]:
      count_norvig += 1
      print(all_corr_sent[i][j])
print("Norvig accuracy", count_norvig/50)

his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat
his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat
his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat
instead of spring my plan the council decided to sell the site to a property developer who will knock it down and build a load of luxury apartments
instead of accepting my plan the council decided to sell the site to a property developer who will knock it down and build a load of luxury apartments
instead of accepting my plan the council decided to sell the site to a property developer who will knock it down and build a load of luxury apartments
this definition of descriptive language could even be used in your everyday speech
this definition of descriptive language could even be used in your everyday speech
thi

In [ ]:
print(norvig_corrected_sentences)

[['my most valuable possession is an old slightly warped blood guitar', 'my most valuable possession is an old slightly warped blood guitar', 'my most valuable possession is an old slightly warped blown guitar', 'my most valuable possession is an old slightly warped blood guitar', 'my most valuable possession is an old slightly warped blown guitar'], ['his paid however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'his paid however does not extend to his appearance for he spends most of his time indoors watching television and growing fat'

In [ ]:
print(converted_orig_sentences)

['my most valuable possession is an old slightly warped blond guitar', 'his pride however does not extend to his appearance for he spends most of his time indoors watching television and growing fat', 'notice how many different descriptors go into just the sentence about how the cat walks', 'the positioning of items relative to other items is on full display in this paragraph to give people a clear vision of the layout of the place as a whole', 'observe how the writer moves clearly from a description of the head of the clown to the body to the unicycle underneath', 'the blond wood has been chipped and gouged to gray particularly where the pick guard fell off years ago', 'instead of the perfect summer day of five minutes ago the sky was darker than the blackest night not only had the sun disappeared there werent even any stars', 'instead of accepting my plan the council decided to sell the site to a property developer who will knock it down and build a load of luxury apartments', 'this 

As you can see, for Norvig solution accuracy on my test cases is 0.28, while my checker obtains 0.56 (easy, I am better than Director of Research in Google). However, this is due to the reason that I do not correct "right" words, while Norvig solution tries to check every word. However, this is not the worst result and better dataset and further improvements should be used to evaluate this work deeper.

## Justification section

1. I decided to use bigrams, because while looking through different N-grams I see that it is most frequent situation that such N gram from the test texts is not presented in the N gram probability tables. Of course, this is also the case for the bigrams, but it is less frequent. I assign the 1e-8 value for such cases. It allows to equally treat the cases for the unknown bigrams and operate with the QWERTY distance between the buttons on the keyboard (typos check)

2. I decided to divide the value obtained by a euclidian distance between the buttons so that do not allow cases where one word chosen instead another just due to the reason that it is "common" typo. For example, if we type "gyu" it is pretty obvious for humans that we want to say "guy", not "buy". But, without any penalties for euclidian distance my program thinks that that "buy" is better than "guy" in such cases (and context doesn't help)

3. Now let me say about bigrams itself. They are collected by group of smart people, however they are not really good and, maybe, requires a deep preprocessing to use. There are a lot of strange words in this bigrams with length 1-3 and they just make noise while predicting the candidate. They obtain scores, they are not so far from the word that we need to correct, and, therefore, while using k = 30 as number of candidates to generate out of these 30 candidates only about 5 are words that are really used in the lexicon. I tried different approaches, for example, nltk words, but, unfortunately, they face exactly the same issue, and, additionally, doesn't have the plural form of words, while these bigrams have. Additionally, I used 5grams, test it and decided to use only bigrams, because it is almost impossible to catch the context of 5 words so that it would be presented in the probability table. Or, another issue, for fivegrams probabilities would be usually 1 or 0, very rarely something between, what is also bad, because in such cases we pay to much attention to the 5gram presented in the dataset, but not necessary we want use it.

Thank you very much for such a beatiful and interesting assignment! This is really sad, that due to the lots of other work I did not create something more interested and accurate system, but I obtain a general knowledge of how spell checker may be implemented in our mobile phones or other auto-correction systems!




  #### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)